In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import mobilenet_v3_small
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
import os


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "./modified-dataset"
num_classes = 10
batch_size = 64
num_epochs = 30


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.25),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

for split in ["train", "val", "test"]:
    path = os.path.join(data_dir, split)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing folder: {path}")
    else:
        print(f"✅ Found: {path}")

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_test_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


✅ Found: ./modified-dataset\train
✅ Found: ./modified-dataset\val
✅ Found: ./modified-dataset\test


In [4]:
model = mobilenet_v3_small(weights="IMAGENET1K_V1")
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\anush/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:04<00:00, 2.50MB/s]


In [5]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)


In [6]:
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(val_dataset)
    val_acc = val_correct / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_mobilenetv3.pth")


Epoch 1/30 Train Loss: 1.9601 Acc: 0.4163 | Val Loss: 1.5231 Acc: 0.6267
Epoch 2/30 Train Loss: 1.2655 Acc: 0.7562 | Val Loss: 1.0965 Acc: 0.7733
Epoch 3/30 Train Loss: 0.9230 Acc: 0.8583 | Val Loss: 0.8854 Acc: 0.8767
Epoch 4/30 Train Loss: 0.8262 Acc: 0.8862 | Val Loss: 0.8027 Acc: 0.9033
Epoch 5/30 Train Loss: 0.7693 Acc: 0.9154 | Val Loss: 0.7603 Acc: 0.9367
Epoch 6/30 Train Loss: 0.7397 Acc: 0.9254 | Val Loss: 0.7159 Acc: 0.9567
Epoch 7/30 Train Loss: 0.7032 Acc: 0.9421 | Val Loss: 0.7029 Acc: 0.9633
Epoch 8/30 Train Loss: 0.6922 Acc: 0.9458 | Val Loss: 0.6982 Acc: 0.9567
Epoch 9/30 Train Loss: 0.6692 Acc: 0.9575 | Val Loss: 0.6874 Acc: 0.9533
Epoch 10/30 Train Loss: 0.6603 Acc: 0.9596 | Val Loss: 0.6831 Acc: 0.9567
Epoch 11/30 Train Loss: 0.6504 Acc: 0.9608 | Val Loss: 0.6798 Acc: 0.9600
Epoch 12/30 Train Loss: 0.6326 Acc: 0.9746 | Val Loss: 0.6733 Acc: 0.9567
Epoch 13/30 Train Loss: 0.6264 Acc: 0.9762 | Val Loss: 0.6715 Acc: 0.9500
Epoch 14/30 Train Loss: 0.6184 Acc: 0.9825 | Va